In [1]:
import cv2
import face_recognition
import numpy as np


In [2]:
# Global variables
known_faces = []
known_names = []
exit_flag = False  # Global exit flag


In [12]:
def detect_and_recognize_faces(video_source=0, window_name="Face Detection and Recognition", width=800, height=600, resize_factor=0.5, frame_skip=2):
    """
    Optimized face detection and recognition for faster video processing.

    :param video_source: Index of the video source (default: 0 for webcam) or path to a video file.
    :param window_name: Name of the OpenCV window.
    :param width: Width of the display window.
    :param height: Height of the display window.
    :param resize_factor: Factor by which to downscale the frames for faster processing (default: 0.5).
    :param frame_skip: Number of frames to skip between processing (default: 2).
    """
    global exit_flag
    exit_flag = False  # Reset the flag when the function starts

    # Create a named window and set its size
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, width, height)

    video_capture = cv2.VideoCapture(video_source)

    if not video_capture.isOpened():
        print("Error: Could not open video source.")
        return

    print("Press 'q' to exit the video stream.")
    frame_count = 0
    while True:
        if exit_flag:  # Checking exit flag
            break

        ret, frame = video_capture.read()
        if not ret:
            print("End of video or failed to capture frame.")
            break

        frame_count += 1

        # Skip frames for faster processing
        if frame_count % frame_skip != 0:
            continue

        # Resize the frame
        small_frame = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)

        # Convert the frame to RGB
        rgb_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # Detect faces
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            name = "Unknown"
            if known_faces:
                # Compare face encoding to known faces
                matches = face_recognition.compare_faces(known_faces, face_encoding)
                face_distances = face_recognition.face_distance(known_faces, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_names[best_match_index]

            # Scale the face location back to the original frame size
            top = int(top / resize_factor)
            right = int(right / resize_factor)
            bottom = int(bottom / resize_factor)
            left = int(left / resize_factor)

            # Draw rectangle around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display frame
        cv2.imshow(window_name, frame)

        # Exit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

In [13]:
def add_known_face(image_path, name):
    """
    Add a known face to the database.
    :param image_path: Path to the image file.
    :param name: Name of the person in the image.
    """
    global known_faces, known_names

    # Load the image
    image = face_recognition.load_image_file(image_path)
    face_encodings = face_recognition.face_encodings(image)

    if face_encodings:
        # Add face encoding and name to the lists
        known_faces.append(face_encodings[0])
        known_names.append(name)
        print(f"Added {name} to known faces.")
    else:
        print("No face detected in the image. Please try again.")


#add_known_face("path/to/image.jpg", "Person Name")


In [14]:
def exit_video():
    """
    Sets the exit flag to terminate video capture.
    """
    global exit_flag
    exit_flag = True
    print("Exit signal received. Closing video stream.")


In [15]:
        #            To detect faces using the webcam
#                                detect_and_recognize_faces(0, width=1280, height=720)

#            To detect faces in a video file
#                               Replace 'path/to/video.mp4' with the actual path to the video file
detect_and_recognize_faces("D:/Work,Courses & Studies/Yomna - edits & Work/CodSoft_Tasks/AI/Task 5/test.mp4",  resize_factor=0.6, frame_skip=2)


Press 'q' to exit the video stream.
